# Segunda Tarefa do Titanic


In [59]:
import pandas as pd
#train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


#### Teste de tratamento das características

In [60]:
# # Pessoas com idade < 14 classificada como crianças
# X['Age'].fillna(X['Age'].mean(), inplace=True)
# test['Age'].fillna(X['Age'].mean(), inplace=True)

# X.loc[ X['Age'] < 14, 'Sex'] = 'children'
# X['Age'] = X['Age'].astype(int)
# test.loc[ test['Age'] < 14, 'Sex'] = 'children'
# test['Age'] = test['Age'].astype(int)

# # Define classe para o sexo
# # Classes: 1: female, 2: male
# X['Sex'] = X['Sex'].map({'male':2, 'female':1, 'children':0})
# test['Sex'] = test['Sex'].map({'male':2, 'female':1, 'children':0})


# # # Define classes para as idades
# # X.loc[ X['Age'] <= 16, 'Age'] = 0
# # X.loc[(X['Age'] > 16) & (X['Age'] <= 32), 'Age'] = 1
# # X.loc[(X['Age'] > 32) & (X['Age'] <= 48), 'Age'] = 2
# # X.loc[(X['Age'] > 48) & (X['Age'] <= 64), 'Age'] = 3
# # X.loc[ X['Age'] > 64, 'Age']
# # test.loc[ test['Age'] <= 16, 'Age'] = 0
# # test.loc[(test['Age'] > 16) & (test['Age'] <= 32), 'Age'] = 1
# # test.loc[(test['Age'] > 32) & (test['Age'] <= 48), 'Age'] = 2
# # test.loc[(test['Age'] > 48) & (test['Age'] <= 64), 'Age'] = 3
# # test.loc[ test['Age'] > 64, 'Age']

# # Classifica pessoas viajando sozinhas e cria nova característica
# X['alone'] = 0
# X.loc[(X['SibSp'] == 0) & (X['Parch'] == 0), 'alone'] = 1
# X.loc[(X['SibSp'] > 0) | (X['Parch'] > 0), 'alone'] = 0
# X['alone'] = X['alone'].astype(int)
# test['alone'] = 0
# test.loc[(test['SibSp'] == 0) & (test['Parch'] == 0), 'alone'] = 1
# test.loc[(test['SibSp'] > 0) | (test['Parch'] > 0), 'alone'] = 0
# test['alone'] = test['alone'].astype(int)

# # # # Define classes para a tarifa
# # X['Fare'].fillna(X['Fare'].median(), inplace = True)
# # test['Fare'].fillna(test['Fare'].median(), inplace = True)

# # X.loc[ X['Fare'] <= 7, 'Fare'] = 0
# # X.loc[(X['Fare'] > 7) & (X['Fare'] <= 15), 'Fare'] = 1
# # X.loc[(X['Fare'] > 15) & (X['Fare'] <= 35), 'Fare']   = 2
# # X.loc[ X['Fare'] > 36, 'Fare'] = 3
# # test.loc[ test['Fare'] <= 7, 'Fare'] = 0
# # test.loc[(test['Fare'] > 7) & (test['Fare'] <= 15), 'Fare'] = 1
# # test.loc[(test['Fare'] > 15) & (test['Fare'] <= 35), 'Fare']   = 2
# # test.loc[ test['Fare'] > 35, 'Fare'] = 3


# # Define classes para Embarked
# #data2['Age'].fillna(data['Age'].median(), inplace=True)
# X['Embarked'] = X['Embarked'].map({'C': 1, 'Q': 2, 'S': 3})
# X['Embarked'].fillna(X['Embarked'].mode()[0], inplace=True)
# test['Embarked'] = test['Embarked'].map({'C': 1, 'Q': 2, 'S': 3})
# test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)



In [61]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        #self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin', 'SibSp', 'Parch']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)
        return Xdrop


In [62]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [63]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [64]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('scaler', StandardScaler()),
    ('classificador', RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [True, False],
    'classificador__max_depth': [5],
}
modelo = GridSearchCV(pipetotal, param_grid=parametros, n_jobs=-1)
scores = cross_validate(modelo, X, y, cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

(array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]),
 nan,
 nan)